# BOTorch tutorial. Fairness MOBO.
Here we illustrate how to tune two machine learning hyper-parameters using BOTorch. In this notebook we will compare BO's performance with respect to RS.

First we install BOTorch

In [ ]:
!pip install botorch

Import libraries. We will just tune the alpha regularization term of a MLPClassifier, the number of neurons in the hidden layer and the momentum value on a synthetic classification problem.

In [ ]:
import os
import torch
import numpy as np
import plotly
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score

seed=1
np.random.seed(seed)
torch.manual_seed(seed)

Objective function: Estimation of the generalization error measured by the lower confidence bound of the accuracy loss function of a multilayer perceptron whose weights are estimated via a k-fold cross validation on a synthetic dataset of 20 variables $\mathbf{X}$ to perform binary classification of the dummy variable $y$. We do not need to shuffle the dataset as it is synthetic, but watch out of that for real problems.

In [ ]:
X, y = make_classification(n_samples=2000, random_state=1)

We have a low fidelity of the real performance brought by this classifier here as I limit the number of epochs of the MLP to 150 for didactic purposes. Also, the variance of the classifier can be improved by setting a higher number of folds, but I also set it to 3 for didactic purposes. We set the synthetic problem to be common for all the experiments. Our two objectives will be inversely correlated, one minimizes the accuracy and the other the true positive rate, usually used for fairness to protect the weak class value (gender, race, disability, religion, etc...). 

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate

def confusion_matrix_scorer(clf, X, y):
      y_pred = clf.predict(X)
      cm = confusion_matrix(y, y_pred)
      return {'tpr': (cm[1, 1])/(cm[0,0] + cm[0,1] + cm[1,0] + cm[1,1]), 'accuracy': (cm[0,0] + cm[1,1])/(cm[0,0] + cm[0,1] + cm[1,0] + cm[1,1])}

def target_function(hyperparameter_sample, seed=1, fidelity="low"):
  #For more clarity, I declare a variable for each hyper-parameter, although it is a begginer move. 
  if fidelity == "low":
    max_epochs = 200
    folds = 3
  else: 
    max_epochs = 500
    folds = 10
  alpha_regularizer = float(hyperparameter_sample[0])
  momentum_value = float(hyperparameter_sample[1])
  hidden_layer_size_value = int(hyperparameter_sample[2])
  X, y = make_classification(n_samples=1000, random_state=1)
  clf = MLPClassifier(random_state=seed, max_iter=max_epochs, alpha=alpha_regularizer, momentum=momentum_value, hidden_layer_sizes=(hidden_layer_size_value))
  scores = cross_validate(clf, X, y, scoring=confusion_matrix_scorer, cv=folds)
  obj_function_1 = scores['test_accuracy']
  obj_function_2 = scores['test_tpr']
  obj_function_1 = obj_function_1.mean() - obj_function_1.std()
  obj_function_2 = obj_function_2.mean() - obj_function_2.std()
  result = []
  result.append(obj_function_1)
  result.append(obj_function_2)
  print("Estimation of the accuracy and TPR of the MLP on the synthetic dataset done")
  return result

Print the accuracy function of the alpha hyper-parameter, that we want to maximize. Here we set the resolution to 100 for didactic purposes. As the 3-fold CV estimates three models to estimate the value of the accuracy parameter that represents the generalization error, we are really training 300 models here. So this may be computationally costly. The other hyper-parameters are set by default, to see the whole function over the input space we would need a 3D visualization. We could also consider as alternative objectives fit_time y score_time to obtain a compromise between faster models or more accurate models. First element is accuracy, second is TPR. We can see how they are inversely correlated.

In [ ]:
import plotly.graph_objects as go

lower_bound = 0.000001
upper_bound = 0.1
resolution = 10
n_objectives = 2
#resolution = 100
x = np.linspace(lower_bound, upper_bound, resolution)
x_new = x.reshape((resolution,-1))
z = np.array([target_function(np.array([alpha, 0.9, 20])) for alpha in x]).reshape((resolution, n_objectives))
accuracy = z[:,0]
tpr = z[:,1]
data1 = go.Scatter(x=x, y=accuracy, line_color="#FE73FF", name="Accuracy")
data2 = go.Scatter(x=x, y=tpr, line_color="#FE73FF", name="TPR")

fig1 = go.Figure(data=data1)
fig2 = go.Figure(data=data2)
fig1.update_layout(title="Accuracy, estimated by 3-fold-CV", xaxis_title="Alpha regularizer value", yaxis_title="Accuracy")
fig1.show()
fig2.update_layout(title="TPR estimated by 3-fold-CV", xaxis_title="Alpha regularizer value", yaxis_title="TPR")
fig2.show()

We now set the bounds for the optimization. First one is the range of the alpha hyper-parameter, then the range for the momentum hyper-parameter and, lastly, the range of the number of neurons of the MLP. Our input space is hence a cube contained in $\mathbb{R}^3$.

In [ ]:
bounds = np.array([[0.000001, 0.8, 10],[0.1, 0.999, 300]])

Generate some data. Example of 1000 random points from the input space, normalized into the range. We can see that their values lie in the range.

In [ ]:
train_x = torch.rand(1000, len(bounds[0])) * (bounds[1] - bounds[0]) + bounds[0]
print(torch.min(train_x, axis=0))
print(torch.max(train_x, axis=0))

We can observe a random sample of the hyper-parameter uniform distribution. 

In [ ]:
train_x[0]

Then we compute the latent function $f(\mathbf{x})$. The true evaluation would be contaminated. $\mathbf{y} = f(\mathbf{x}) + \epsilon \quad s.t. \quad \epsilon \approx N(0, \sigma) \quad f: \mathbb{R}^3 \to \mathbb{R}^2$

In [ ]:
exact_obj = target_function(train_x[0], seed=seed)
exact_obj

We wrap all of the previous code into a function to be used freely

In [ ]:
def generate_initial_data(n=10):
  train_x = torch.rand(n, len(bounds[0])) * (bounds[1] - bounds[0]) + bounds[0]
  exact_obj = torch.tensor([target_function(sample) for sample in train_x])
  best_observed_value = exact_obj.max().item()
  return train_x, exact_obj, best_observed_value

In [ ]:
generate_initial_data(3)

Let us now invoke this function to start the BO iteration.

In [ ]:
init_x, init_y, best_init_y = generate_initial_data(3)

We now declare a function that give us a suggestion with respect to the GP fitted to previous points.

In [ ]:
init_y.shape

In [ ]:
from botorch.optim import optimize_acqf
from botorch.utils.transforms import normalize, unnormalize
from botorch.models.transforms.outcome import Standardize
from botorch.models import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement
from botorch import fit_gpytorch_model
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions.dominated import DominatedPartitioning

def optimize_qehvi_and_get_observation(model, train_x, train_obj, sampler, bounds, reference_point, normalized_bounds):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    
    with torch.no_grad():
        pred = model.posterior(normalize(train_x, bounds)).mean
    partitioning = FastNondominatedPartitioning(
        ref_point=reference_point, 
        Y=pred,
    )
    acq_func = qExpectedHypervolumeImprovement(
        model=model,
        ref_point=reference_point,
        partitioning=partitioning,
        sampler=sampler,
    )
    
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=normalized_bounds,
        q=1,
        num_restarts=200,
        raw_samples=512,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
        sequential=True,
    )
    # observe new values 
    new_x =  unnormalize(candidates.detach(), bounds=bounds)
    new_obj = target_function(new_x[0])
    return new_x, new_obj
    
def get_next_points(init_x, init_y, bounds, reference_point, normalized_bounds, n_points=1):
  train_x = normalize(init_x, bounds)
  models = []
  for i in range(init_y.shape[1]):
    train_y = init_y[..., i:i+1]
    models.append(SingleTaskGP(train_x, train_y, outcome_transform=Standardize(m=1)))
  model = ModelListGP(*models)
  mll = SumMarginalLogLikelihood(model.likelihood, model)
  fit_gpytorch_model(mll)
  qehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([128]))
  
  configuration, observation = optimize_qehvi_and_get_observation(model, init_x, init_y, qehvi_sampler, bounds, reference_point, normalized_bounds)

  return configuration, observation

Now we will compare the performance of RS vs BO. We just do, for didactic purposes, 3 iterations with different random seed of each method. In order to do it better, we will need 30 iterations of both methods and then perform a t-test of 2 means statistical hypotheses test. We save the performance in each iteration of the methods. Both methods have an initial random observation.

In [ ]:
n_experiments = 10
n_iterations= 50
n_methods = 2

#We store the results in this tensor, we just have two methods to compare.
experiment_results = np.zeros((n_methods, n_experiments, n_iterations))
experiment_configurations = np.zeros((n_methods, n_experiments, n_iterations, len(bounds[0])))
normalized_bounds = torch.tensor([np.zeros(len(bounds[0])), np.ones(len(bounds[0]))])
bounds = np.array([[0.000001, 0.8, 10.0],[0.1, 0.999, 300.0]])
reference_point = torch.tensor([0.7, 0.3])

for i in range(n_experiments):
  np.random.seed(i)
  torch.manual_seed(i)

  init_x, init_y, best_init_y = generate_initial_data(1)

  candidates=[]
  results=[]
  
  best_observed_result_bo = DominatedPartitioning(ref_point=reference_point, Y=init_y).compute_hypervolume().item()
  best_observed_candidate_bo = init_x[0]

  random_sample_hyperparameters = torch.rand(1, len(bounds[0])) * (bounds[1] - bounds[0]) + bounds[0]
  
  best_observed_result_rs = DominatedPartitioning(ref_point=reference_point, Y=torch.tensor([target_function(random_sample_hyperparameters[0])])).compute_hypervolume().item()
  best_observed_candidate_rs = random_sample_hyperparameters[0]

  for j in range(n_iterations):

    #BO Code.
    new_candidates, new_results = get_next_points(init_x, init_y, bounds, reference_point, normalized_bounds)

    print(f"New candidates are: {new_candidates}")
    init_x = torch.cat([init_x, new_candidates])
    init_y = torch.cat([init_y, torch.tensor([new_results])])

    # compute hypervolume
    bd = DominatedPartitioning(ref_point=reference_point, Y=init_y)
    volume = bd.compute_hypervolume().item()
        
    if best_observed_result_bo < volume:
      best_observed_result_bo = volume
      best_observed_candidate_bo = new_candidates[0]

    experiment_configurations[0,i,j,:] = best_observed_candidate_bo
    experiment_results[0,i,j] = best_observed_result_bo
    
    ##RS Code.
    #I sample the hyperparameter value for random search here.
    random_sample_hyperparameters = torch.rand(1, len(bounds[0])) * (bounds[1] - bounds[0]) + bounds[0]
    rs_obj_fun_result = DominatedPartitioning(ref_point=reference_point, Y=torch.tensor([target_function(random_sample_hyperparameters[0])])).compute_hypervolume().item()
    
    if best_observed_result_rs < rs_obj_fun_result:
      best_observed_result_rs = rs_obj_fun_result
      best_observed_candidate_rs = random_sample_hyperparameters[0]
    experiment_configurations[1,i,j,:] = best_observed_candidate_rs
    experiment_results[1,i,j] = best_observed_result_rs
    print("Current best BO candidate: " + str(best_observed_candidate_bo))
    print("Current best RS candidate: " + str(best_observed_candidate_rs))
    print("Current best BO hypervolume: " + str(best_observed_result_bo))
    print("Current best RS hypervolume: " + str(best_observed_result_rs))
    print(f"Number of iterations done: {(j+1)}")
    print(f"Number of experiment: {(i+1)}")

We give the recommendation as the best observed result. 

In [ ]:
best_observed_result = np.max(experiment_results)
index_set = np.where(experiment_results==best_observed_result)
print("The best observed result is: " + str(best_observed_result))
print("The best observed result belong to the : " + str(index_set[0][0]) + " method. Its value is " + str(experiment_configurations[index_set][0]))

And now we plot the results to compare both methods.

In [ ]:
import plotly.graph_objects as go

x = np.linspace(1, n_iterations, n_iterations).astype(int)
mean_bo = np.mean(experiment_results[0,:,:], axis=0)
mean_rs = np.mean(experiment_results[1,:,:], axis=0)
std_bo = np.std(experiment_results[0,:,:], axis=0)
std_rs = np.std(experiment_results[1,:,:], axis=0)
bo_ub_results = go.Scatter(x=x, y=mean_bo + 0.5*std_bo, mode='lines', name="", line_color="green", line_width=0.1)
bo_results = go.Scatter(x=x, y=mean_bo, mode='lines', fill='tonexty', line_color="green", name="Bayesian Optimization")
bo_lb_results = go.Scatter(x=x, y=mean_bo - 0.5*std_bo, mode='lines', fill='tonexty', name="", line_color="green", line_width=0.1)

rs_ub_results = go.Scatter(x=x, y=mean_rs + 0.5*std_rs, mode='lines', name="", line_color="red", line_width=0.1)
rs_results = go.Scatter(x=x, y=mean_rs, mode='lines', fill='tonexty', line_color="red", name="Random Search")
rs_lb_results = go.Scatter(x=x, y=mean_rs - 0.5*std_rs, mode='lines', fill='tonexty', name="", line_color="red", line_width=0.1)
  
fig = go.Figure()
fig.add_trace(bo_ub_results)
fig.add_trace(bo_results)
fig.add_trace(bo_lb_results)
fig.add_trace(rs_ub_results)
fig.add_trace(rs_results)
fig.add_trace(rs_lb_results)
fig.update_layout(title="Performance comparison between BO and RS. MLP full experiment.", xaxis_title="Iterations", yaxis_title="Accuracy lower bound")
fig.show()